In [4]:
!pip install nlp
!pip install transformers


     |████████████████████████████████| 778kB 5.3MB/s 
     |████████████████████████████████| 890kB 27.1MB/s 
     |████████████████████████████████| 1.1MB 39.7MB/s 
     |████████████████████████████████| 3.0MB 40.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d4ee6445dd8e63a169583088c51ee543e61daed156b7205648d3108c1a5c2471
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
!git clone https://github.com/chrisliu298/tapt.git

Cloning into 'tapt'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 785 (delta 177), reused 245 (delta 88), pack-reused 447
Receiving objects: 100% (785/785), 147.77 MiB | 21.54 MiB/s, done.
Resolving deltas: 100% (386/386), done.
Checking out files: 100% (67/67), done.


In [2]:
import os
os.chdir("/content/tapt/src")

In [8]:
import pandas as pd
from pprint import pprint
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from utils.data_pipeline import prepare_data
from utils.data_pipeline import prepare_custom_data
from utils.metrics import compute_metrics


def tokenize(batch):
    """Tokenize a batch of data (with padding and truncation).

    Arg:
        batch: A batch of training data.
    """
    return tokenizer(
        batch["text"], padding="max_length", truncation=True, max_length=512
    )

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base")

# Load dataset
train_dataset, val_dataset, test_dataset = prepare_data(
    tokenize_func=tokenize,
    dataset_name="yelp_polarity",
    train_count=10,
    train_size=5,
    val_size=5,
    use_all_test=False,
    test_count=10,
    test_size=5,
    others=5,
    seed=42,
)
# Load custom data
augmented = prepare_custom_data(
    tokenize_func=tokenize, dataset_name="/content/nlp_yelp_train.tsv"
)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl

AttributeError: ignored